In [31]:
workspace = 'dev'
collection = "EdFi9"
version = "5.2"
options = {'header':False}

In [32]:
%run /OEATestKit_py

In [33]:
def should_test(entity_path):
    #Should only run test on data if there is data to test=
    folders = oea.get_folders(entity_path)
    if len(folders) == 0:
        return False
    batch_type, source_data_format = oea.get_batch_info(entity_path)
    source_url = oea.to_url(f'{entity_path}/{batch_type}_batch_data')
    if batch_type == 'snapshot' or batch_type=='additive': source_url = f'{source_url}/{oea.get_latest_folder(source_url)}'
    return oea.get_folder_size(source_url) > 0
    
def validate_all_data_was_ingested(tables_source, path_modifier):
    items = oea.get_folders(tables_source)
    options = {'header':False}
    for item in items: 
        try:
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                if(should_test(f'{tables_source}/{item}')):
                    oea_test_kit.test_raw_data_is_subset_of_lake(collection, version, item, 'id', options, path_modifier)
            #print(f"item {item} passed test")
        except AnalysisException as e:
            print(f"item {item} failed test")

def validate_no_duplicates(table_source, path_modifier):
    items = oea.get_folders(table_source)
    for item in items: 
        try:
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                if(should_test(f'{table_source}/{item}')):
                    oea_test_kit.test_has_duplicates(collection, version, item, 'id', path_modifier)
            #print(f"item {item} passed test")
        except AnalysisException as e:
            print(f"item {item} failed test")

In [34]:
validate_all_data_was_ingested(f'stage1/Transactional/{collection}/{version}/ed-fi','ed-fi')
validate_all_data_was_ingested(f'stage1/Transactional/{collection}/{version}/tpdm','tpdm')

In [35]:
validate_no_duplicates(f'stage1/Transactional/{collection}/{version}/ed-fi','ed-fi')
validate_no_duplicates(f'stage1/Transactional/{collection}/{version}/tpdm','tpdm')